# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import logging
import os
import json
import csv
import numpy as np
import pandas as pd
import joblib

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.core import Model, Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core.conda_dependencies import CondaDependencies

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

experiment_name = 'hyperdrive-exp1'

experiment=Experiment(ws, experiment_name)

In [ ]:
found = False
key = "heartdisease"
description_text = ""

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

df = dataset.to_pandas_dataframe()
df.describe()

In [ ]:
hd_cluster = 'hyperdrive-cluster'
try:
    compute_target = ComputeTarget(workspace=ws, name=hd_cluster)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS12_V2',
                                                           vm_priority = 'lowpriority', 
                                                           max_nodes=5)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)
    print('Cluster created successfully.')

compute_target.wait_for_completion()

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=2)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling(
    {
        '--C' : choice(0.001,0.01,0.1,1.0,10.0,100.0),
        '--max_iter': choice(25,50)
    }
)

#TODO: Create your estimator and hyperdrive config
estimator = ScriptRunConfig(source_directory = ".",
                            script='train.py',
                            arguments=['--data', df],
                            compute_target=hd_cluster,
                            environment=environment)

hyperdrive_run_config = HyperDriveConfig(run_config=estimator,
                                     hyperparameter_sampling=param_sampling, 
                                     policy=early_termination_policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=24,
                                     max_concurrent_runs=4)

In [ ]:
#TODO: Submit your experiment
run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(run).show()

In [ ]:
run.wait_for_completion()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = run.get_best_run_by_primary_metric()
best_run.get_file_names()

print("Best Model Accuracy:", best_automl.get_metrics()['accuracy'])
print("Best Model Type: ", best_automl.get_details()['properties']['run_algorithm'])

In [ ]:
#TODO: Save the best model
joblib.dump(model, 'hd_model.joblib')
joblib.load('hd_model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
environment = Environment.get(ws,"AzureML-HyperDrive")

model = best_run.register_model(model_name='hd-best-model', model_path='outputs/model.pkl')
inference_config = InferenceConfig(entry_script='score.py',
                                   environment=environment)
service_name = 'hd-deploy'
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1.8, memory_gb=4)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deployment_config,
                       overwrite=True
                      )
service.wait_for_deployment(show_output=True)

scoring_uri = service.scoring_uri
print(scoring_uri)
service.update(enable_app_insights=True)
service.wait_for_deployment(show_output=True)


TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
data = {"data": [{"age": 63,
        "sex": 1,
        "cp": 3,
        "trestbps": 145,
        "chol": 233,
        "fbs": 1,
        "restecg": 0,
        "thalach": 150,
        "exang": 0,
        "oldpeak": 2.3,
        "slope": 0,
        "ca": 0,
        "thal": 1},

    {"age": 56,
        "sex": 1,
        "cp": 0,
        "trestbps": 125,
        "chol": 249,
        "fbs": 1,
        "restecg": 0,
        "thalach": 144,
        "exang": 1,
        "oldpeak": 1.2,
        "slope": 1,
        "ca": 1,
        "thal": 2},
      ]}
    
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
# headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())
print("Case 0: No Heart Disease, Case 1: Heart Disease.")

# test using service instance
output = service.run(input_data)
output



TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
logs = service.get_logs()
logs

In [ ]:
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

